In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
def normalizeDataset(path):
    #schema mediato finale di schema alignment
    df = pd.read_csv(path, sep=';')

    for column in df.columns:
        df[column] = df[column].astype(str).str.upper()
    return df[df.columns[1:]]


In [ ]:
df_integration = pd.read_csv('output_integration/final_plt.csv')

In [ ]:
import time
directory = 'datasets_for_augmentation'
dataset_list = Path(directory).glob('*.csv')

matches = {'500Companies.csv': [('0', 'stock'), ('3', 'sector'),('4', 'address')],
            '400Companies.csv': [('0', 'stock'), ('2', 'sector')],
            'AirlineCodes.csv': [('2', 'country')]
         }

i=0
num_substitutions = 0
num_error = 0
start = time.time()
for path in dataset_list:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = normalizeDataset(path)
    name_dataset = str(path).split('\\')[1]
    try:
      for index, row in df_integration.iterrows():
         for index2, row2 in df2.iterrows():
            if len(row['name']) > 3:
               if row['name'] in row2['1']:
                  for col_df2, col_integration in matches[name_dataset]:
                     if df_integration[df_integration.index == index][col_integration].isnull().values.any():
                        num_substitutions+=1
                        row.at[col_integration] = row2[col_df2]
    except Exception as e:
      print(e)
      print("ERRORE " + str(path))
      num_error = num_error + 1

print('Num Substitutions: ' + str(num_substitutions))
print('Num Errors: ' + str(num_error))
print('Time Elapsed: ' + str(time.time() - start))
df_integration